In [2]:
from sys import path

path.append("../")

import jax
import jax.numpy as jnp
import flax
from flax.struct import dataclass
from flax import linen as nn

In [8]:
linear = nn.Dense(5)
linear.variables

ValueError: Can't access variables on unbound modules